# Dark Matter Capture

by <br />
**Adam Green** (*UC Riverside*, [agree019@ucr.edu](mailto:agree019@ucr.edu)) <br />
**Anton Panis** (*North Hollywood HGM*, [aopanis@gmail.com](mailto:aopanis@gmail.com))<br />
**Flip Tanedo** (*UC Riverside*, [flip.tanedo@ucr.edu](mailto:flip.tanedo@ucr.edu))<br />

## Introduction

### Brief introduction to dark matter and dark sectors

*To be written by Adam and Anton* 

### Capture in the Earth and annihilation into dark photons

We study the following process:
1. Dark matter is captured in the Earth
2. Dark matter annihilates into dark photons in the center of the Earth
3. The dark photons pass through the earth and decay near the surface
4. These dark photons may be detected using the IceCube experiment

### What this code does

This notebook calculates each of the following steps given a definition of a dark matter model: the mass of the dark matter $m_X$ , the dark photon interaction strength $\alpha_X$, and the *kinetic mixing* $\varepsilon$ that controls how much the dark photon talks to ordinary matter.

### Initialize

In [2]:
import numpy as np
import scipy as sp
import pandas
from scipy import interpolate

import scipy.integrate as integrate
import matplotlib.pyplot as plt
print 'Complete'

Complete


### Importing and Interpolating Earth Density Data

In [3]:
%matplotlib inline

# Radius is column 1, Density is column 2
# Radius in meters: verified by REF [47] Table 1 pg. 308
# Density in grams/cm^3, verified by REF [47] Table 1 pg. 308
# For me later on: n_N.iat[i,j] # Access the element at the 'ith' row and 'jth' column of n_N starting at (i,j) = (0,0)



# This section grabs the data and converts it to MKS units
data = pandas.read_csv('PREM500.csv', sep = ',')
radiusTemp1 = data[[0]]  # Radius in Meters
density_badunits = data[[1]] # Density in g/cm^3
densityTemp1 = density_badunits * (100)**3 / 1000 # Density in kg/m^3
n_N = data[['Radius', 'Density']]


# The interpolation function doesn't like these objects, so they need to be massaged into 1-D numpy arrays
radius = np.asarray(radiusTemp1).squeeze()
density = np.asarray(densityTemp1).squeeze()


# Here are the two interpolations of the density data

# Linear Interpolation
n_NLinear = sp.interpolate.interp1d(radius, density, kind='linear')
print 'Earth density data is stored in n_NLinear'

# Curvy Interpolation
#tck = sp.interpolate.splrep(radius, density) #Error: possibly because 'density' isn't an actual funciton, but a list of numbers
#xnew = range(0, 10)
#ynew = interpolate.splev(xnew, tck)

print 'Complete'

Earth density data is stored in n_NLinear
Complete


### Atomic Definitions and Conversions

In [4]:
################################################################################
# Conversions
################################################################################

def amu2Gev(par1):
    return 0.9314941 * par1 # GeV

def amu2g(par1):
    return 1.66053892*10**-24 * par1 # grams

def GeV2s(par1):
    return 1.52*10**24 * par1 # s^-1

def s2GeV(par1):
    return 1.52*10**24 * par1 # GeV^-1

def GeV2cm(par1):
    return 5.06*10**13 * par1 # cm^-1

def cm2GeV(par1):
    return 5.06*10**13 * par1 # GeV^-1

def KeltoGeV(par1):
    return 8.62*10**-14 * par1 # GeV

def s2yr(par1):
    return 3.16888*10**-8 * par1 # Yr



################################################################################
# Atomic Definitions
################################################################################
# Access dictionary values as: dictionaryName['key']
# Dictionaries are sorted by value as:
    # for key, value in sorted(dicName.iteritems(), key=lambda (k,v): (v,k)):

# Dictionaries must be called as:

    # def testfunction(element):
    #     return atomicNumbers[element]
    # print testfunction('H1')

atomicNumbers = { # This is 'A' just below eqn 10
    'H1': 1.,
    'He4': 4.,
    'He3': 3.,
    'C12': 12.,
    'C13':13.,
    'N14':14.,
    'N15':15.,
    'O16':16.,
    'O17':17 ,
    'O18':18.,
    'Ne20':20.,
    'Na':23.,
    'Mg24':24., # 78%
    'Al27':27.,
    'Si28':28.,
    'P31':31.,
    'S32':32.,
    'Cl35':35., # 75%
    'Ar40':40.,
    'K39':39.,
    'Ca40':40.,
    'Sc45':45.,
    'Ti48':48., # 74%
    'V51':51.,
    'Cr52':52., # 83%
    'Mn55':55.,
    'Fe56':56.,
    'Co59':59.,
    'Ni58':58. # 58%
}

isotopicMasses = { # This is m_N anywhere you see it
    'H1':1.007825,
    'He4':4.0026,
    'He3':3.0160,
    'C12':12.,
    'C13':13.003355,
    'N14':14.003074,
    'N15':15.000109,
    'O16':15.994915,
    'O17':16.999132 ,
    'O18':17.99916,
    'Ne20':19.992440,
    'Na':22.989770,
    'Mg24':23.985042, # 78%
    'Al27':26.981538,
    'Si28':27.976927,
    'P31':30.973762,
    'S32':31.972071,
    'Cl35':34.99688, # 75%
    'Ar40':39.962383,
    'K39':38.963707,
    'Ca40':39.962591,
    'Sc45':44.95591,
    'Ti48':47.947947, # 74%
    'V51':50.943964,
    'Cr52':51.940512, # 83%
    'Mn55':54.938050,
    'Fe56':55.934942,
    'Co59':58.9332,
    'Ni58':57.935348, # 58%
}

nProtons = { # This is Z_N
    'H1':1,
    'He3':2,
    'He4':2,
    'He3':2,
    'C12':6,
    'C13':6,
    'N14':7,
    'N15':7,
    'O16':8,
    'O17':8,
    'O18':8,
    'Ne20':10,
    'Na':11,
    'Mg24':12, # 78%
    'Al27':13,
    'Si28':14,
    'P31':15,
    'S32':16,
    'Cl35':17, # 75%
    'Ar40':18,
    'K39':19,
    'Ca40':20,
    'Sc45':21,
    'Ti48':22, # 74%
    'V51':23,
    'Cr52':24, # 83%
    'Mn55':25,
    'Fe56':26,
    'Co59':27,
    'Ni58':28 # 58%
}

# Mass Fraction Dictionary
coreMassFrac = {
    'O16' : 0.009,
    'Mg24': 0.0,
    'Al27': 0.0,
    'Si28': 0.06,
    'P31' : 0.002,
    'S32' : 0.019,
    'Ca40': 0.0,
    'Cr52': 0.009,
    'Fe56': 0.855,
    'Ni58': 0.052    
}

mantleMassFrac = {
    'O16' : 0.440,
    'Mg24': 0.228,
    'Al27': 0.0235,
    'Si28': 0.210,
    'P31' : 0.00009,
    'S32' : 0.00025,
    'Ca40': 0.0253,
    'Cr52': 0.0026,
    'Fe56': 0.0626,
    'Ni58': 0.00196    
}
print 'Complete'

Complete


### Model Parameters and Constants

In [5]:
################################################################################
# Model Parameters
################################################################################

# Do I even need these guys?
'''
 Get these values from Flip
e =
'''

# Input by hand for now
#print '''Input Model Parameters: m_X, m_A, epsilon, alpha, alpha_X: '''
global m_X
global m_A
global epsilon
global alpha
global alpha_X

print'''Assuming the following model parameters: 
m_X = 1
m_A = 1
epsilon = 1
alpha = 1
alpha_X = 1'''

m_X = 1.
m_A = 1.
epsilon = 1.
alpha = 1.
alpha_X = 1.

#m_X = float(raw_input("m_X = "))
#m_A = float(raw_input("m_A = "))
#epsilon = float(raw_input("epsilon = "))
#alpha = float(raw_input("alpha = "))
#alpha_X = float(raw_input("alpha_X = "))

assert m_X >= 0, 'Dark matter mass is negative'
assert m_A >= 0, 'Dark photon mass is negative'
assert epsilon >= 0, 'Coupling constant epsilon is negative'
assert alpha >= 0, 'Coupling constant alpha is negative'
assert alpha_X >= 0, 'Coupling constant alpha_X is negative'

################################################################################
# Constants
################################################################################

global G
global M_E
global R_earth
global V_dot
global V_cross
global V_gal
global u_0
global k
global n_X
global mf

G = 6.674*10**-11 # N m^2/ kg^2
M_E = 5.972 * 10**24 # kg
R_earth = 6371000 # m      # Changed from R_earth = 6370000 m via google search
R_crit = 3480000 # m
V_dot = 220000 # m/s
V_cross = 29800 # m/s
V_gal = 550000 # m/s
u_0 = 245000 # m/s
k = 2.5
n_X = 0.3e6/m_X #GeV/m^3   # Changed from n_X = 0.3/m_X GeV/cm^3
mf = 1.0

print 'Complete'

Assuming the following model parameters: 
m_X = 1
m_A = 1
epsilon = 1
alpha = 1
alpha_X = 1
Complete



### Function Definitions

In [6]:
################################################################################
# Function Definitions
################################################################################

# FUTURE CHANGES:
# (Done)  - m_N -> isotopicMasses[key]
# (Done)  - Z_N -> numProtons[key]
# (Done)  - v_cross -> escapeVelocity(r)

def heaviside (arg1):
    if (arg1 >= 0):
         return 1
    elif (arg1 < 0):
        return 0


def eqn09 (r, u, E_R, element, epsilon = epsilon, alpha = alpha, alpha_X = alpha_X, m_A = m_A):
    
    dSigmaNdE_R = 8 * np.pi * epsilon**2 * alpha_X * alpha * nProtons[element]**2 *\
    (isotopicMasses[element]) / ( (u**2 + (escapeVelocity(r))**2) * (2*isotopicMasses[element] * E_R + m_A**2)**2 ) * ( eqn10(E_R, element) )**2
    
    assert dSigmaNdE_R >= 0, 'Input Values: (%e, %e, %e, %s) results in negative dSigmaNdE_R (eqn09)' % (r, u, E_R, element)
    
    return dSigmaNdE_R



def eqn10(E_R, element):
    E_N = 0.114/(isotopicMasses[element]**float(5/3))
    F_N = np.exp(-E_R/ E_N)
    
    assert E_N > 0, 'E_N cannot be negative'
    assert F_N >= 0, 'Input values (%e, %s) results in negative F_N' % (E_R, element)
    
    return F_N


def eqn13Emin(u, m_X = m_X):
    E_min = 0.5 * m_X * u**2
    
    assert E_min >= 0, 'Input value (%e) results in negative E_min' % u
    
    return E_min


def eqn13Emax(r, u, element, m_X = m_X):
    r_cross = escapeVelocity(r)
    mu_N = (isotopicMasses[element] *m_X) / (isotopicMasses[element] + m_X)
    E_max = 2 * mu_N**2 * (u**2 + escapeVelocity(r)**2) / isotopicMasses[element]
    
    assert mu_N > 0, 'Input values (%e, %e, %s) result in negative reduced mass' % (r, u, element)
    assert E_max > 0, 'Input values (%e, %e, %s) result in negative E_max' % (r, u, element)
    
    return E_max


def escapeVelocity(r):
    return np.sqrt(2* G * M_E / r)


def eqn15(element): # for a single element only right now
    r_crit = 3480000 # meters
    
    def integrand (E, u, r):
        assert 0 <= r, 'Input value (%e) results in negative radius' % r
        assert r <= R_earth, 'Input value (%e) results in radius beyond R_earth' % r
        
        if r <= 3480000:
            mf = coreMassFrac[[element]]
        else:
            mf = mantleMassFrac[[element]]
        
        v_cross = escapeVelocity(r)
        eqn09step = eqn09(r, u, E_R, element)
        eqn16step = eqn16(u)
        
        E_max = eqn13Emax(r, u, element)
        E_min = eqn13Emin(u)
        
        assert v_cross > 0, 'Input values (%e, %s) result in negative escape velocity' % (r, element)
        assert eqn09step >= 0, 'Input values (%e, %e, %e, %s) result in negative eqn09step' % (r, u, E, element)
        assert eqn16step > 0, 'Input values (%e, %s) result in negative eqn16step' % (u, element)
               
        assert E_max > 0, 'Input values (%e, %e, %s) result in negative E_max' % (r, u, element)
        assert E_min > 0, 'Input values (%e) result in negative E_min' % u
            
        return n_X * (4*np.pi)**2 * r**2 * n_NLinear(r) * u**2 * eqn16step * ( ( u**2 + (v_cross)**2 ) / u )* eqn09step * heaviside(E_max - E_min) 
       
        # Add some sort of logic control if statement such that if radius is under radius critical = radius separating core and  mantle the code knows to switch to the appropriate mass fraction dictionary
        
    rmin = 0
    rmax = R_earth

    umin = lambda r: 0
    umax = lambda r: np.inf

    emin = lambda r, u: E_min
    emax = lambda r, u: E_max

    return integrate.tplquad( integrand, rmin, rmax, umin, umax, emin, emax)[0]


def eqn16(u1): # Returns only the scalar associated with the value of the integral
    u = u1
    def integrand(x, y, u, V_dot = V_dot, V_cross = V_cross): #x = cos(theta), y = cos(phi)
        return eqn17( ( u**2 + (V_dot+V_cross * y)**2 + 2 * u * (V_dot + V_cross*y) *x)** 0.5  )

    return integrate.dblquad(integrand, -1, 1, lambda y: -1, lambda y: 1, args = (u, V_dot, V_cross))[0]


def normalization17(x):
    arg = ( (V_gal**2 - x**2) / (k * u_0**2) )
    tempVar = (np.exp( arg - 1 )) ** 2.5
    
    assert tempVar >= 0, 'Eqn17 cannot be negative'
    
    return tempVar

alpha = (integrate.quad(normalization17, -V_gal, V_gal))[0]
N_0 = (1./alpha)
assert N_0 > 0, 'Eqn17normalization is less than zero'


def eqn17(u, N_0 = N_0, k = k, u_0 = u_0, V_gal = V_gal):
    fofu = N_0 * ( np.exp ( (V_gal**2 - u**2) / (k * u_0**2) ) - 1)**k * heaviside(V_gal - u)
    
    assert fofu > 0, 'Input values (%e, %e) result in negative DM Velocity distribution' % (u, N_0)
    
    return fofu


print 'Complete'

Complete


## Scratchwork

### Template (sorta) for setting up the weight function n_N(r) dictionary accounting for mass fraction
Still need to implement user-defined input (element -> dictionary key) and pass that into functions as a a key for a dictionary

### Convert this cell to code
####### This is the template for the logic control of the radius

mydict1 = {
    'key1': 1,
    'key2': 3
}

mydict2 = {
    'key1': 2,
    'key2': 4
}


def testfunction():
    
    def integrand(r):
        if r <= 5:
            a = mydict1['key1']
        else:
            a = mydict2['key1']
            
        return r * a
    
    return integrate.quad(integrand, 0, 10)

testfunction()
    

### Getting a tripple Integral to evaluate to a number

### Convert this cell to code

m_N = 1.
m_X = 1.
R_Cross = 6370000

def escapeVelocity(r):
    return np.sqrt(G*M_E / r)

def eqn13Emin(u, m_X = m_X):
    E_min = 0.5 * m_X * u**2
    return E_min

def eqn13Emax(r, u, m_N = m_N, m_X = m_X):
    r_cross = escapeVelocity(r)
    mu_N = (m_N *m_X) / (m_N + m_X)
    
    E_max = 2 * mu_N**2 * (u**2 + r_cross**2) / m_N
    return E_max

def testfunct1():
    def integrand(E, u, r):
        
        Emax = eqn13Emax(r, u)
        Emin = eqn13Emin(u)
        
        return heaviside(Emax - Emin) * r**2 * u**2 
    
    rmin = 0.
    rmax = R_Cross
    
    umin = lambda r: 0.
    umax = lambda r: np.inf
    
    Emin = lambda r,u: eqn13Emin(u)
    Emax = lambda r,u: eqn13Emax(r,u)
    
    return integrate.tplquad( integrand, rmin, rmax, umin, umax, Emin, Emax)

print testfunct1()

### Template for passing 'element' parameter acting as a dictionary key 

### Convert this cell to code

def testfunctiona(element):  
    A_N = isotopicMasses[element]
    # Define a bunch of other stuff dependent on 'element' ouside the integrand
    
    def integrand1(r):
        return A_N * r
    
    return integrate.quad(integrand1, 0, 1)